Stock Trading Bot calculater using Custom CNN model

Introduction

Our facial expression indentification using pre-trained models is a deep-learning application in artificial inteligence. This project proposes to use pre-trained convolutional neural network models to recognize and categorize facial expressions such as angry, disgust, fear, happy, neutral, sad, and surpise. The initiative, which builds on prior research and improvements in transfer learning, seeks to enhance emotion indentification skills in real-world settings ranging from mental health to human-computer systems.

Our Two Key Phases:

Phase 1: Designing and implementing a customized CNN model for facial expression indentification using TensorFlow API, Keras and Gradio.

Phase 2: Experimenting with pre-trained models and having an understanding of transfer learning using TensorFlow API and Keras.

Our Plan: To compare the preformance of our custom-built model to pre-trained models and access their ability to indentify facial expressions.

Project Team:

Hibah Hibah: 169033490

Iman Chaudhry: 210650820

Arrangan Kathir: 169042354

Tyler Rizzi: 169022274

Brandon Dang: 169026034

In [1]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from matplotlib.ticker import FuncFormatter, MaxNLocator
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, LSTM
import gradio as gr
import matplotlib.animation as animation

CNN Model

Phase 1: Our Custom CNN Model Design and Implementation To classify facial expressions, we built a simple CNN model with TensorFlow and Keras API. The model design starts with multiple convolutional layers, then max-pooling layers, next a flattening layer, and lastly dense layers. Our implementation attempts to provide simple knowldge of CNN and its application in image classification problems, specifically for facial expression indentification.

Model Architecture

Conv2D Layers: A convolutional process extracts features from an input picture.

MaxPooling2D Layers: Reduces spatial dimentions of feature maps, lowering computational load and reuducing overfitting.

Flatten Layer: Converts the 2D feature maps to 1D feature vectors in preperation for fully connecting layers.

Dense Layers: Fully connected layers that conduct categorization using the extracted features.

In [2]:
# Function to predict future stock prices using the trained model
def predict_stock_prices(symbol, initial_capital, start_date, end_date):
    global cash, stocks, buy_price, sell_price, final_capital, df, test_df, sequence_length, features, model, buy_count, sell_count

    buy_count = 0
    sell_count = 0

    # Append '.csv' to the symbol to construct the filename
    filename = f"{symbol}.csv"

    # Construct the path to the data file
    data_path = os.path.join('archive', 'stocks', filename)

    # Load the dataset
    df = pd.read_csv(data_path)

    # Ensure correct data types
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    # Automatically determine the start and end dates of the data
    available_start_date = df.index.min()
    available_end_date = df.index.max()

    # Ensure provided dates are within the available range
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    if start_date < available_start_date or end_date > available_end_date:
        raise ValueError(f"Date range out of bounds. Available range is from {available_start_date.date()} to {available_end_date.date()}")

    # Use the determined date range
    df = df[start_date:end_date]

    # Feature columns and target
    features = ['Open', 'High', 'Low', 'Close', 'Volume']
    target = 'Close'

    # Check if the subset is empty
    if df.empty:
        raise ValueError("DataFrame is empty. Check the date range and ensure the data file contains data for the specified period.")

    # Separate scaler for the 'Close' column
    close_scaler = MinMaxScaler()

    # Normalize the Close prices
    df['Close'] = close_scaler.fit_transform(df[['Close']])

    # Normalize other features
    scaler = MinMaxScaler()
    df[features] = scaler.fit_transform(df[features])

    # Create sequences (e.g., using past 30 days to predict the next day)
    sequence_length = 30

    def create_sequences(data, seq_length, features, target):
        sequences = []
        labels = []
        for i in range(len(data) - seq_length):
            sequences.append(data.iloc[i:i + seq_length][features].values)
            labels.append(data.iloc[i + seq_length][target])
        return np.array(sequences), np.array(labels)

    # Split the data into training and testing sets
    train_size = int(len(df) * 0.8)
    train_df = df[:train_size]
    test_df = df[train_size:]

    # Check if the test data is too small, and adjust accordingly
    if len(test_df) <= sequence_length:
        train_df = df
        test_df = df
        sequence_length = max(1, int(len(df) * 0.2))

    # Create sequences
    X_train, y_train = create_sequences(train_df, sequence_length, features, target)
    X_test, y_test = create_sequences(test_df, sequence_length, features, target)

    # Check the shapes of the data
    print(f"X_train shape: {X_train.shape}")
    print(f"y_train shape: {y_train.shape}")
    print(f"X_test shape: {X_test.shape}")
    print(f"y_test shape: {y_test.shape}")

    # Ensure the test data is not empty
    if X_test.shape[0] == 0 or y_test.shape[0] == 0:
        raise ValueError("Test data is empty. Increase the size of the dataset or adjust the sequence length.")

    # Define the CNN model
    model = Sequential([
        Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(sequence_length, len(features))),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=32, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(20, activation='relu'),
        Dense(1)
    ])

    model.compile(optimizer='adam', loss='mse')
    model.summary()

    # Train the model
    model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

    # Reset cash and stocks
    cash = initial_capital
    stocks = 0
    buy_dates = []
    sell_dates = []
    buy_price = None
    sell_price = None

    # Initialize logs
    logs = []
    capital_changes = [f"Initial Capital: ${cash}"]

    # Generate predictions
    df['Predicted_Close'] = np.nan
    for i in range(len(test_df) - sequence_length):
        X_input = test_df.iloc[i:i + sequence_length][features].values
        X_input = X_input.reshape((1, sequence_length, len(features)))
        pred = model.predict(X_input)[0][0]
        df.iloc[train_size + sequence_length + i, df.columns.get_loc('Predicted_Close')] = pred

    # Denormalize the predictions and actuals for accurate plotting
    df['Close'] = close_scaler.inverse_transform(df[['Close']])
    df['Predicted_Close'] = close_scaler.inverse_transform(df[['Predicted_Close']])

    # Initialize data for animation
    dates = []
    close_prices = []
    predicted_prices = []
    buys = {'dates': [], 'prices': []}
    sells = {'dates': [], 'prices': []}

    # Define the buy and sell logic
    df['Min'] = df['Low'][df['Low'] == df['Low'].rolling(window=3, center=True).min()]
    df['Max'] = df['High'][df['High'] == df['High'].rolling(window=3, center=True).max()]

    fig, ax1 = plt.subplots(figsize=(14, 7))
    fig.suptitle('Stock Prices with Buy/Sell Signals', fontsize=16)

    # Plot the closing prices and predicted closing prices
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Price')
    date_form = DateFormatter("%Y-%m")
    ax1.xaxis.set_major_formatter(date_form)

    # Add grid lines
    ax1.grid(True, linestyle='--', alpha=0.5)

    # Plot the data with more details
    line, = ax1.plot(df.index, df['Close'], label='Close Prices', color='blue', linewidth=1)
    pred_line, = ax1.plot(df.index, df['Predicted_Close'], label='Predicted Close Prices', linestyle='--', color='orange', linewidth=1)
    buy_scatter = ax1.scatter([], [], marker='^', color='green', label='Buy Signal', s=100)
    sell_scatter = ax1.scatter([], [], marker='v', color='red', label='Sell Signal', s=100)

    # Plot moving averages
    df['SMA9'] = df['Close'].rolling(window=9).mean()
    df['SMA21'] = df['Close'].rolling(window=21).mean()
    ax1.plot(df.index, df['SMA9'], label='SMA 9', color='purple', linewidth=1)
    ax1.plot(df.index, df['SMA21'], label='SMA 21', color='cyan', linewidth=1)

    ax1.legend(loc='upper left')

    # Create a secondary y-axis for the volume bars
    ax2 = ax1.twinx()
    ax2.bar(df.index, df['Volume'], color='blue', alpha=0.3, width=1)
    ax2.grid(False)

    # Set y-axis limits for Volume and Price
    ax1.set_ylim(df['Close'].min(), df['Close'].max())
    ax2.set_ylim(df['Volume'].min(), df['Volume'].max())

    # Format x-axis to show only the major ticks and labels
    ax1.xaxis.set_major_locator(MaxNLocator(nbins=8))

    # Improve y-axis formatting for better readability
    ax1.yaxis.set_major_formatter('${x:,.2f}')

    ax2.get_yaxis().set_visible(False)

    # Function to format volume axis
    def volume_formatter(x, pos):
        if x >= 1e6:
            return f'{x*1e-6:.1f}M'
        elif x >= 1e3:
            return f'{x*1e-3:.1f}K'
        else:
            return f'{x:.0f}'

    ax2.yaxis.set_major_formatter(FuncFormatter(volume_formatter))

    def animate(i):
        global cash, stocks, buy_price, sell_price, final_capital, buy_count, sell_count

        if i >= len(df):
            return

        current_date = df.index[i]
        current_low = df['Low'].iloc[i]
        current_high = df['High'].iloc[i]
        current_close = df['Close'].iloc[i]
        current_pred = df['Predicted_Close'].iloc[i]

        dates.append(current_date)
        close_prices.append(current_close)
        predicted_prices.append(current_pred)

        if not np.isnan(df['Min'].iloc[i]):
            if cash > 0:
                buy_price = current_low
                stocks = cash / buy_price
                cash = 0
                buys['dates'].append(current_date)
                buys['prices'].append(current_close)
                log = f"Bought at {buy_price:.2f} on {current_date.date()}"
                logs.append(log)
                buy_count += 1

        if not np.isnan(df['Max'].iloc[i]):
            if stocks > 0:
                sell_price = current_high
                cash = stocks * sell_price
                stocks = 0
                sells['dates'].append(current_date)
                sells['prices'].append(current_close)
                log = f"Sold at {sell_price:.2f} on {current_date.date()}"
                logs.append(log)
                sell_count += 1
                capital_changes.append(f"Capital on {current_date.date()}: ${cash:.2f}")

        final_capital = cash + stocks * current_close

        line.set_data(dates, close_prices)
        pred_line.set_data(dates, predicted_prices)
        if buys['dates']:
            buy_scatter.set_offsets(np.c_[buys['dates'], buys['prices']])
        if sells['dates']:
            sell_scatter.set_offsets(np.c_[sells['dates'], sells['prices']])
        ax1.relim()
        ax1.autoscale_view()
        ax2.relim()
        ax2.autoscale_view()

        return line, pred_line, buy_scatter, sell_scatter

    ani = animation.FuncAnimation(fig, animate, frames=len(df), interval=1000, repeat=False)  # Adjust interval for frame rate
    ani.save('trading_animation.gif', writer='pillow', fps=1)  # Adjust fps for realistic animation

    logs.append(f"Final Capital: ${final_capital:.2f}")
    capital_changes.append(f"Final Capital: ${final_capital:.2f}")
    order_summary = f"Total Buy Orders: {buy_count}\nTotal Sell Orders: {sell_count}"

    return 'trading_animation.gif', '\n'.join(logs), '\n'.join(capital_changes), order_summary




Data Preperation

For this project, we use the FER-2013 dataset, which is availabe on Kaggle. This dataset contains 35,887 grayscale images of faces, labeled with one of the seven expressions: happy, sad, angry, surprised, fear, disgust, or neutral. Each picture is a 48x48 pixels in size.

Our Steps for Preperation

Step 1: Dataset Loading: Loading FER-2013 dataset from Kaggle and uploading our path names to Jupyter Notebook.

Step 2: Dataset Preprocessing: Preprocessing the images by adjusting their pixel values and resizing them to our models required input size.

Step 3: Data Augumentation: Using data augumentation to boost the variety of our training data which will prevent any overfitting.

Step 4: Data Splitting: To analyze our models preformance, the dataset is divided in 3 sets these will be training, validation, and testing.

In [3]:
def get_available_date_range(symbol):
    filename = f"{symbol}.csv"
    data_path = os.path.join('archive', 'stocks', filename)
    df = pd.read_csv(data_path)
    df['Date'] = pd.to_datetime(df['Date'])
    available_start_date = df['Date'].min().date()
    available_end_date = df['Date'].max().date()
    return available_start_date, available_end_date

def update_date_range(symbol):
    available_start_date, available_end_date = get_available_date_range(symbol)
    return f"Available date range for {symbol}: {available_start_date} to {available_end_date}"

def main(symbol, initial_capital, start_date, end_date):
    date_range = update_date_range(symbol)
    gif, logs, capital_changes, order_summary = predict_stock_prices(symbol, initial_capital, start_date, end_date)
    return date_range, gif, logs, capital_changes, order_summary



In [4]:
# Get the list of available CSV files
available_symbols = [f.split('.')[0] for f in os.listdir('archive/stocks') if f.endswith('.csv')]

with gr.Blocks() as iface:
    symbol_input = gr.Dropdown(label="Stock Ticker Symbol", choices=available_symbols, interactive=True, allow_custom_value=True)
    date_range_output = gr.Textbox(label="Available Date Range", interactive=False)
    initial_capital_input = gr.Number(label="Initial Capital", value=10000, step=100)
    start_date_input = gr.Textbox(label="Start Date (YYYY-MM-DD)")
    end_date_input = gr.Textbox(label="End Date (YYYY-MM-DD)")
    order_summary_output = gr.Textbox(label="Order Summary", interactive=False)
    gif_output = gr.Image(type="filepath")
    logs_output = gr.Textbox(label="Logs", interactive=False)
    capital_changes_output = gr.Textbox(label="Capital Changes", interactive=False)

    symbol_input.change(
        fn=update_date_range,
        inputs=symbol_input,
        outputs=date_range_output
    )

    predict_button = gr.Button("Predict")
    predict_button.click(
        fn=main,
        inputs=[symbol_input, initial_capital_input, start_date_input, end_date_input],
        outputs=[date_range_output, gif_output, logs_output, capital_changes_output, order_summary_output]
    )


ChatBot

In [5]:
# Chatbot function
def stock_chatbot(message, history):
    # Simple response logic for the chatbot
    if "predict" in message.lower():
        return "You can predict stock prices using the tool above!"
    elif "zs" in message.lower():
        return "ZS is thi stock"
    else:
        return "I'm here to help with your stock queries!"

# Modify your existing gr.Blocks layout
with gr.Blocks() as iface:
    with gr.Tab("Stock Prediction"):
        symbol_input = gr.Dropdown(label="Stock Ticker Symbol", choices=available_symbols, interactive=True, allow_custom_value=True)
        date_range_output = gr.Textbox(label="Available Date Range", interactive=False)
        initial_capital_input = gr.Number(label="Initial Capital", value=10000, step=100)
        start_date_input = gr.Textbox(label="Start Date (YYYY-MM-DD)")
        end_date_input = gr.Textbox(label="End Date (YYYY-MM-DD)")
        order_summary_output = gr.Textbox(label="Order Summary", interactive=False)
        gif_output = gr.Image(type="filepath")
        logs_output = gr.Textbox(label="Logs", interactive=False)
        capital_changes_output = gr.Textbox(label="Capital Changes", interactive=False)

        symbol_input.change(
            fn=update_date_range,
            inputs=symbol_input,
            outputs=date_range_output
        )

        predict_button = gr.Button("Predict")
        predict_button.click(
            fn=main,
            inputs=[symbol_input, initial_capital_input, start_date_input, end_date_input],
            outputs=[date_range_output, gif_output, logs_output, capital_changes_output, order_summary_output]
        )
    
    with gr.Tab("Chatbot"):
        gr.ChatInterface(stock_chatbot).launch()

iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://e01aa70094380a0ca6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


3 Pretrained CNN models

From Hugging Face 

Restnet50
Inception V3
DenseNet121


3 Pre-Trained Models and Gradio Interface
Phase 2: Experimenting with Pre-Trained Models

We use pre-trained models to recognize facial expressions using transfer learning. For this project, we used fine-tuned models like VGG16, ResNet50, InceptionV3.

VGG16 - https://keras.io/api/applications/vgg/#vgg16-function

ResNet50 - https://keras.io/api/applications/resnet/#resnet50-function

InceptionV3 - https://keras.io/api/applications/inceptionv3/

In [6]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.applications import ResNet50, InceptionV3, DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


Restnet50 Model

In [7]:
#Block 1: Resnet50 Model Integration

# Function to load and preprocess stock data
def load_and_preprocess_stock_data(symbol, start_date, end_date):
    data = pd.read_csv(f'archive/stocks/{symbol}.csv')
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)
    data = data[start_date:end_date]

    # Normalizing the close prices
    scaler = MinMaxScaler()
    data['Close'] = scaler.fit_transform(data[['Close']])

    return data[['Close']], scaler

# Load and preprocess stock data
data, scaler = load_and_preprocess_stock_data('ZS', '2020-01-01', '2020-04-01')
X_train = np.array(data.index).reshape(-1, 1)  # Dates are not directly used but for consistency
y_train = data['Close'].values

# Load ResNet50 base model with ImageNet weights
resnet_base = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# Add custom layers for stock prediction
x = resnet_base.output
x = GlobalAveragePooling2D()(x)
x = Dense(1, activation='linear')(x)

# Final ResNet50 model for stock prediction
resnet_model = Model(inputs=resnet_base.input, outputs=x)
resnet_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('resnet50_stock_model.keras', monitor='val_loss', save_best_only=True)

# Train the model (using synthetic data to match image input requirements)
X_train_synthetic = np.random.rand(len(X_train), 224, 224, 3)  # Generating synthetic data to match image input requirements

history = resnet_model.fit(
    X_train_synthetic, y_train,
    epochs=5,
    validation_split=0.2,
    callbacks=[early_stopping, checkpoint]
)

# Save the trained model
resnet_model.save('resnet50_stock_model.keras')

# Predict stock price for validation
y_pred = resnet_model.predict(X_train_synthetic)

# Inverse transform the predictions and true values to get back to the original scale
y_train = scaler.inverse_transform(y_train.reshape(-1, 1))
y_pred = scaler.inverse_transform(y_pred)

# Calculate regression metrics
mse = mean_squared_error(y_train, y_pred)
mae = mean_absolute_error(y_train, y_pred)
r2 = r2_score(y_train, y_pred)

print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R2 Score: {r2:.2f}")


Epoch 1/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 26s 3s/step - accuracy: 0.0475 - loss: 2.1242 - val_accuracy: 0.0000e+00 - val_loss: 0.5604
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 849ms/step - accuracy: 0.0237 - loss: 4.2166 - val_accuracy: 0.0000e+00 - val_loss: 2.7302
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 850ms/step - accuracy: 0.0237 - loss: 6.6025 - val_accuracy: 0.0000e+00 - val_loss: 863.0571
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 878ms/step - accuracy: 0.0237 - loss: 10.1499 - val_accuracy: 0.0000e+00 - val_loss: 5281.4941
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 879ms/step - accuracy: 0.0237 - loss: 2.1362 - val_accuracy: 0.0000e+00 - val_loss: 937.2711
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step
Mean Squared Error (MSE): 351.21
Mean Absolute Error (MAE): 17.75
R2 Score: -8.36


InceptionV3 Pre-trained Model

In [8]:
# Block 2: InceptionV3 Pre-trained Model Integration

# Function to generate stock data image
def generate_stock_image(symbol, start_date, end_date):
    data = pd.read_csv(f'archive/stocks/{symbol}.csv')
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)
    data = data[start_date:end_date]

    fig, ax = plt.subplots(figsize=(5, 5))
    ax.plot(data.index, data['Close'], label='Close Prices', color='blue')
    ax.axis('off')

    plt.savefig(f'{symbol}_stock_image.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    return f'{symbol}_stock_image.png', data['Close']

# Load and preprocess stock image
def load_and_preprocess_stock_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

# Generate the stock image and get true prices for evaluation
image_path, true_prices = generate_stock_image('ZS', '2020-01-01', '2020-04-01')

# Load InceptionV3 base model with ImageNet weights
inception_base = InceptionV3(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# Add custom layers for stock prediction
x = inception_base.output
x = GlobalAveragePooling2D()(x)
x = Dense(1, activation='linear')(x)

# Final InceptionV3 model for stock prediction
inception_model = Model(inputs=inception_base.input, outputs=x)
inception_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('inceptionv3_stock_model.keras', monitor='val_loss', save_best_only=True)

# Load and preprocess the stock image
X_train = load_and_preprocess_stock_image(image_path)

# Reshape data to match InceptionV3 input shape
X_train = np.repeat(X_train, len(true_prices), axis=0)
y_train = true_prices.values

# Train the model
history = inception_model.fit(
    X_train, y_train,
    epochs=5,
    validation_split=0.2,
    callbacks=[early_stopping, checkpoint]
)

# Save the trained model
inception_model.save('inceptionv3_stock_model.keras')

# Predict stock price for validation
y_pred = inception_model.predict(X_train)

# Calculate regression metrics
mse = mean_squared_error(y_train, y_pred)
mae = mean_absolute_error(y_train, y_pred)
r2 = r2_score(y_train, y_pred)

# Print accuracy and loss
train_acc = history.history['accuracy'][-1]
val_acc = history.history['val_accuracy'][-1]
print(f"Training Accuracy: {train_acc:.2f}")
print(f"Validation Accuracy: {val_acc:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R2 Score: {r2:.2f}")


Epoch 1/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 28s 3s/step - accuracy: 0.0000e+00 - loss: 3001.1682 - val_accuracy: 0.0000e+00 - val_loss: 2564.7166
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 2753.6570 - val_accuracy: 0.0000e+00 - val_loss: 2141.0720
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 2488.9446 - val_accuracy: 0.0000e+00 - val_loss: 1643.2181
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.0000e+00 - loss: 2233.7971 - val_accuracy: 0.0000e+00 - val_loss: 926.3355
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 1936.7411 - val_accuracy: 0.0000e+00 - val_loss: 285.8595
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
Training Accuracy: 0.00
Validation Accuracy: 0.00
Mean Squared Error (MSE): 268.16
Mean Absolute Error (MAE): 15.20
R2 Score: -6.14


DenseNet121 Pre-Trained Model

In [9]:
# DenseNet121 Pre-trained Model Integration

# Function to generate stock data image
def generate_stock_image(symbol, start_date, end_date):
    data = pd.read_csv(f'archive/stocks/{symbol}.csv')
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)
    data = data[start_date:end_date]

    fig, ax = plt.subplots(figsize=(5, 5))
    ax.plot(data.index, data['Close'], label='Close Prices', color='blue')
    ax.axis('off')

    plt.savefig(f'{symbol}_stock_image.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    return f'{symbol}_stock_image.png', data['Close']

# Load and preprocess stock image
def load_and_preprocess_stock_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

# Generate the stock image and get true prices for evaluation
image_path, true_prices = generate_stock_image('ZS', '2020-01-01', '2020-04-01')

# Load DenseNet121 base model with ImageNet weights
densenet_base = DenseNet121(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# Add custom layers for stock prediction
x = densenet_base.output
x = GlobalAveragePooling2D()(x)
x = Dense(1, activation='linear')(x)

# Final DenseNet121 model for stock prediction
densenet_model = Model(inputs=densenet_base.input, outputs=x)
densenet_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('densenet121_stock_model.keras', monitor='val_loss', save_best_only=True)

# Load and preprocess the stock image
X_train = load_and_preprocess_stock_image(image_path)

# Reshape data to match DenseNet121 input shape
X_train = np.repeat(X_train, len(true_prices), axis=0)
y_train = true_prices.values

# Train the model
history = densenet_model.fit(
    X_train, y_train,
    epochs=5,
    validation_split=0.2,
    callbacks=[early_stopping, checkpoint]
)

# Save the trained model
densenet_model.save('densenet121_stock_model.keras')

# Predict stock price for validation
y_pred = densenet_model.predict(X_train)

# Calculate regression metrics
mse = mean_squared_error(y_train, y_pred)
mae = mean_absolute_error(y_train, y_pred)
r2 = r2_score(y_train, y_pred)

# Print accuracy and loss
train_acc = history.history['accuracy'][-1]
val_acc = history.history['val_accuracy'][-1]
print(f"Training Accuracy: {train_acc:.2f}")
print(f"Validation Accuracy: {val_acc:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R2 Score: {r2:.2f}")



Epoch 1/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 45s 4s/step - accuracy: 0.0000e+00 - loss: 3254.0696 - val_accuracy: 0.0000e+00 - val_loss: 3442.1479
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 967ms/step - accuracy: 0.0000e+00 - loss: 3060.1018 - val_accuracy: 0.0000e+00 - val_loss: 3813.0574
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 950ms/step - accuracy: 0.0000e+00 - loss: 2799.6553 - val_accuracy: 0.0000e+00 - val_loss: 4477.3213
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 957ms/step - accuracy: 0.0000e+00 - loss: 2591.4573 - val_accuracy: 0.0000e+00 - val_loss: 5007.7427
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 940ms/step - accuracy: 0.0000e+00 - loss: 2423.7822 - val_accuracy: 0.0000e+00 - val_loss: 5462.2422
1/2 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/stepWARNING:tensorflow:6 out of the last 6 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x00000165BCA9D9E0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) cr

Converts from a GIF to an MP4

In [10]:
from PIL import Image
import cv2
import os

# Function to extract frames from GIF
def extract_frames(gif_path, output_folder):
    gif = Image.open(gif_path)
    frame_count = 0

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    while True:
        frame_path = os.path.join(output_folder, f"frame_{frame_count:03d}.png")
        gif.save(frame_path, 'PNG')
        frame_count += 1

        try:
            gif.seek(gif.tell() + 1)
        except EOFError:
            break

    return output_folder, frame_count

# Function to create MP4 from frames
def create_video_from_frames(output_folder, output_video_path, fps=10):
    images = [img for img in os.listdir(output_folder) if img.endswith(".png")]
    images.sort()

    if not images:
        raise ValueError("No frames found in the output folder")

    # Determine the width and height from the first image
    first_frame = cv2.imread(os.path.join(output_folder, images[0]))
    height, width, layers = first_frame.shape

    video = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    for image in images:
        frame = cv2.imread(os.path.join(output_folder, image))
        video.write(frame)

    video.release()

# Paths
gif_path = './trading_animation.gif'
output_folder = 'gif_frames'
output_video_path = 'trading_animation.mp4'

# Extract frames
extract_frames(gif_path, output_folder)

# Create video
create_video_from_frames(output_folder, output_video_path)

print(f"Conversion complete: {output_video_path}")


Conversion complete: trading_animation.mp4


Runs all the pre trained models on gradio

In [11]:
from PIL import Image
import cv2
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import gradio as gr
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from matplotlib.ticker import MaxNLocator, FuncFormatter

# Function to extract frames from GIF
def extract_frames(gif_path, output_folder):
    gif = Image.open(gif_path)
    frame_count = 0

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    while True:
        frame_path = os.path.join(output_folder, f"frame_{frame_count:03d}.png")
        gif.save(frame_path, 'PNG')
        frame_count += 1

        try:
            gif.seek(gif.tell() + 1)
        except EOFError:
            break

    return output_folder, frame_count

# Load the pre-trained models
resnet_model = load_model('resnet50_stock_model.keras')
inception_model = load_model('inceptionv3_stock_model.keras')
densenet_model = load_model('densenet121_stock_model.keras')

# Function to calculate final capital based on predictions
def calculate_final_capital(predictions, initial_capital):
    cash = initial_capital
    stocks = 0
    for i in range(1, len(predictions)):
        predicted_price = predictions[i]
        if predicted_price > predictions[i-1]:  # Buy condition
            if cash > 0:
                stocks = cash / predicted_price
                cash = 0
        elif predicted_price < predictions[i-1]:  # Sell condition
            if stocks > 0:
                cash = stocks * predicted_price
                stocks = 0
    final_capital = cash + stocks * predictions[-1]
    return final_capital

# Function to predict stock prices with ensemble of pre-trained models
def predict_stock_prices_with_ensemble(symbol, initial_capital, start_date, end_date):
    # Load and prepare the data
    filename = f"{symbol}.csv"
    data_path = os.path.join('archive', 'stocks', filename)
    df = pd.read_csv(data_path)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    df = df[start_date:end_date]

    # Normalize the Close prices
    close_scaler = MinMaxScaler()
    df['Close'] = close_scaler.fit_transform(df[['Close']])

    predictions = {
        "ResNet50": [],
        "InceptionV3": [],
        "DenseNet121": [],
        "Ensemble": []
    }

    output_folder, frame_count = extract_frames(gif_path, 'gif_frames')

    for i in range(frame_count):
        frame_path = os.path.join(output_folder, f"frame_{i:03d}.png")
        img = cv2.imread(frame_path)
        if img is None:
            raise ValueError(f"Could not load image from {frame_path}")
        frame_resized = cv2.resize(img, (224, 224))
        frame_array = np.expand_dims(frame_resized, axis=0) / 255.0

        # Predict with each model
        resnet_pred = resnet_model.predict(frame_array)[0][0]
        inception_pred = inception_model.predict(frame_array)[0][0]
        densenet_pred = densenet_model.predict(frame_array)[0][0]

        # Store predictions
        predictions["ResNet50"].append(resnet_pred)
        predictions["InceptionV3"].append(inception_pred)
        predictions["DenseNet121"].append(densenet_pred)

    # Combine predictions by averaging
    predictions["Ensemble"] = np.mean([predictions["ResNet50"], predictions["InceptionV3"], predictions["DenseNet121"]], axis=0)

    # Denormalize the predictions
    df['Predicted_Close'] = np.nan
    df['Predicted_Close'].iloc[-len(predictions["Ensemble"]):] = close_scaler.inverse_transform(np.array(predictions["Ensemble"]).reshape(-1, 1)).flatten()

    # Plot the results
    fig, ax1 = plt.subplots(figsize=(14, 7))
    fig.suptitle('Stock Prices with Pre-Trained Model Predictions', fontsize=16)

    # Plot the closing prices and predicted closing prices
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Price')
    date_form = DateFormatter("%Y-%m")
    ax1.xaxis.set_major_formatter(date_form)

    # Add grid lines
    ax1.grid(True, linestyle='--', alpha=0.5)

    # Plot the actual and predicted prices
    ax1.plot(df.index, close_scaler.inverse_transform(df[['Close']]), label='Actual Close Prices', color='blue', linewidth=1)
    ax1.plot(df.index, df['Predicted_Close'], label='Predicted Close Prices', linestyle='--', color='orange', linewidth=1)

    # Highlight the Min and Max prices
    min_price = df['Close'].min()
    max_price = df['Close'].max()
    min_date = df['Close'].idxmin()
    max_date = df['Close'].idxmax()
    
    ax1.annotate(f"Min: ${min_price:.2f}", xy=(min_date, min_price), xytext=(min_date, min_price),
                 textcoords="offset points", xycoords='data', arrowprops=dict(facecolor='green', shrink=0.05),
                 ha='right', va='bottom')
    ax1.annotate(f"Max: ${max_price:.2f}", xy=(max_date, max_price), xytext=(max_date, max_price),
                 textcoords="offset points", xycoords='data', arrowprops=dict(facecolor='red', shrink=0.05),
                 ha='right', va='bottom')

    ax1.legend(loc='upper left')

    # Set y-axis limits for Price
    ax1.set_ylim(close_scaler.inverse_transform(np.array([df['Close'].min(), df['Close'].max()]).reshape(-1, 1)).flatten())

    # Format x-axis to show only the major ticks and labels
    ax1.xaxis.set_major_locator(MaxNLocator(nbins=8))

    # Improve y-axis formatting for better readability
    ax1.yaxis.set_major_formatter('${x:,.2f}')

    plt.show()

    # Calculate final capitals
    resnet_capital = calculate_final_capital(predictions["ResNet50"], initial_capital)
    inception_capital = calculate_final_capital(predictions["InceptionV3"], initial_capital)
    densenet_capital = calculate_final_capital(predictions["DenseNet121"], initial_capital)
    ensemble_capital = calculate_final_capital(predictions["Ensemble"], initial_capital)

    final_capital_str = f"Final Capital using Ensemble Model: ${ensemble_capital:.2f}"
    return gif_path, resnet_capital, inception_capital, densenet_capital, final_capital_str

# Define the function to get available date range
def get_available_date_range(symbol):
    filename = f"{symbol}.csv"
    data_path = os.path.join('archive', 'stocks', filename)
    df = pd.read_csv(data_path)
    df['Date'] = pd.to_datetime(df['Date'])
    available_start_date = df['Date'].min().date()
    available_end_date = df['Date'].max().date()
    return available_start_date, available_end_date

# Function to update date range based on selected symbol
def update_date_range(symbol):
    available_start_date, available_end_date = get_available_date_range(symbol)
    return f"Available date range for {symbol}: {available_start_date} to {available_end_date}"

# Main function that integrates everything
def main(symbol, initial_capital, start_date, end_date):
    date_range = update_date_range(symbol)
    gif, resnet_capital, inception_capital, densenet_capital, final_capital = predict_stock_prices_with_ensemble(symbol, initial_capital, start_date, end_date)
    
    resnet_capital_str = f"ResNet50 Model Capital: ${resnet_capital:.2f}"
    inception_capital_str = f"InceptionV3 Model Capital: ${inception_capital:.2f}"
    densenet_capital_str = f"DenseNet121 Model Capital: ${densenet_capital:.2f}"
    
    return date_range, gif, resnet_capital_str, inception_capital_str, densenet_capital_str, final_capital

# Get the list of available CSV files
available_symbols = [f.split('.')[0] for f in os.listdir('archive/stocks') if f.endswith('.csv')]

# Gradio interface
with gr.Blocks() as iface:
    symbol_input = gr.Dropdown(label="Stock Ticker Symbol", choices=available_symbols, interactive=True, allow_custom_value=True)
    date_range_output = gr.Textbox(label="Available Date Range", interactive=False)
    initial_capital_input = gr.Number(label="Initial Capital", value=10000, step=100)
    start_date_input = gr.Textbox(label="Start Date (YYYY-MM-DD)")
    end_date_input = gr.Textbox(label="End Date (YYYY-MM-DD)")
    gif_output = gr.Image(type="filepath")
    resnet_capital_output = gr.Textbox(label="ResNet50 Model Capital", interactive=False)
    inception_capital_output = gr.Textbox(label="InceptionV3 Model Capital", interactive=False)
    densenet_capital_output = gr.Textbox(label="DenseNet121 Model Capital", interactive=False)
    final_capital_output = gr.Textbox(label="Final Capital (Ensemble Model)", interactive=False)

    symbol_input.change(
        fn=update_date_range,
        inputs=symbol_input,
        outputs=date_range_output
    )

    predict_button = gr.Button("Predict")
    predict_button.click(
        fn=main,
        inputs=[symbol_input, initial_capital_input, start_date_input, end_date_input],
        outputs=[date_range_output, gif_output, resnet_capital_output, inception_capital_output, densenet_capital_output, final_capital_output]
    )

iface.launch(share=True)


Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://1c102a082c34946e2f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 947ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━

C:\Users\Ranjot Sandhu\AppData\Local\Temp\ipykernel_55936\2029320891.py:104: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['Predicted_Close'].iloc[-len(predictions["Ensemble"]):] = close_scaler.inverse_transform(np.array(predictions["Ense